In [1]:
from sklearn import preprocessing
import matplotlib.pyplot as plt
from copy import deepcopy
import pandas as pd
import numpy as np
import importlib
import sklearn
import random
import pickle
import torch
import math

import socialSig
importlib.reload(socialSig)
from helpers import *

In [79]:
####### Load our Data
#y - 'number_moved'
#x - 'everything else that is or can be represented as a float.'
devSet = pd.read_csv("./us_migration.csv")
devSet = devSet.loc[:, ~devSet.columns.str.contains('^Unnamed')]
devSet = devSet.apply(lambda x: pd.to_numeric(x, errors='coerce'))
devSet = devSet.dropna(axis=1)
devSet = devSet.drop(['sending'], axis = 1)

y = torch.Tensor(devSet['US_MIG_05_10'].values)
X = devSet.loc[:, devSet.columns != "US_MIG_05_10"].values

mMScale = preprocessing.MinMaxScaler()
X = mMScale.fit_transform(X)

In [80]:
model = socialSig.SocialSigNet(X=X, outDim = 1)


In [81]:
checkpoint = torch.load("./trained_weights_nosending4.torch")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [43]:
devSet.loc[:, devSet.columns != "US_MIG_05_10"]#.values

,sending,sending_citizen_unspecified,sending_citizenship_unknown,sending_household_not_owned,sending_household_owned,sending_household_owned_unknown,sending_indigeneity,sending_internet,sending_internet_unknown,sending_marriage_unknown,...,sending_total_pop,sending_unknown_employment_status,sending_unknown_indigeneity,sending_unpaid_worker,sending_urban,sending_weighted_avg_income,sending_weighted_avg_income_abroad,sending_weighted_avg_no_income_abroad,sending_weighted_avg_unknown_income_abroad,sending_widowed
0,10001,1134.995083,243.827586,1178.209016,1106.832815,736.562500,918.960526,2111.233685,347.698113,576.362069,...,59606,1218.357143,840.339623,0,1597.239059,1124.200953,768.788518,1213.505104,1931.825000,460.430041
1,10002,917.067746,600.000000,1274.160656,795.118020,-1.000000,-1.000000,1988.103175,459.166667,342.800000,...,4750,535.750000,576.379310,0,-1.000000,908.782316,720.767123,929.317603,845.567568,745.377359
2,10003,637.977315,557.150000,729.454795,610.791414,468.392857,584.454545,1310.462428,514.312500,3800.000000,...,4420,540.000000,629.215385,0,-1.000000,628.585520,486.116034,738.533015,-1.000000,276.569767
3,10004,1178.378744,1678.562500,1560.244328,1084.109969,785.742857,1165.943463,2109.431851,1043.500000,549.695652,...,35742,771.400000,694.579439,0,1436.934837,1173.751889,861.385645,1219.547797,1643.137255,741.363985
4,10005,2231.128863,2916.538705,2104.943216,2278.834178,2353.519005,2527.268790,3075.771459,1941.578834,650.206897,...,657314,1531.622086,2064.311411,0,2319.113608,2230.022635,1672.340116,2273.630651,3946.180995,1241.236982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,9002,2405.699630,1748.869065,2219.106684,2552.982875,1890.022296,2330.599992,3055.875149,1689.971097,1135.496873,...,2242270,2519.015098,2299.741654,0,2406.749527,2404.611760,2001.012098,2415.147145,2104.591252,1416.154574
2318,9003,3858.707011,1231.109956,2915.200938,4484.619480,5818.942935,2810.099863,5217.779456,2357.150198,214.283803,...,241966,3975.823333,2876.307321,0,3855.573265,3849.228718,3068.219990,3900.477908,1393.715692,2411.162593
2319,9004,1927.404919,784.350000,1570.619453,2037.162477,3061.779412,1854.212578,2597.090254,1346.494048,283.783784,...,140362,1364.578313,2710.145782,0,1906.936589,1925.807227,2464.181193,1922.865676,594.935000,1399.683105
2320,9005,2289.519646,1626.816777,2062.935985,2430.503155,2294.108333,2340.950516,2822.150244,139.097744,667.244318,...,360724,2594.140756,2834.652051,0,2294.638487,2286.092043,2303.242828,2289.790755,1463.121810,1253.617190


In [39]:
devSet.loc[:, devSet.columns != "US_MIG_05_10"].values[0]

array([1.00010000e+04, 1.13499508e+03, 2.43827586e+02, 1.17820902e+03,
       1.10683281e+03, 7.36562500e+02, 9.18960526e+02, 2.11123369e+03,
       3.47698113e+02, 5.76362069e+02, 1.92162072e+03, 1.14339273e+03,
       9.51874055e+02, 6.77628459e+01, 7.96264417e+02, 4.33637241e+03,
       4.60644907e+03, 1.60599141e+03, 4.48376764e+02, 1.19929500e+07,
       5.96060000e+04, 1.21835714e+03, 8.40339623e+02, 0.00000000e+00,
       1.59723906e+03, 1.12420095e+03, 7.68788518e+02, 1.21350510e+03,
       1.93182500e+03, 4.60430041e+02])

In [29]:
torch.tensor(X[0])

tensor([2.8984e-01, 1.5086e-01, 1.9041e-02, 1.7205e-01, 1.3949e-01, 1.1063e-02,
        8.4842e-02, 9.4775e-02, 1.1623e-02, 5.2897e-02, 1.5080e-01, 1.4954e-01,
        1.6302e-01, 4.8031e-04, 1.9931e-02, 2.4111e-01, 9.3632e-02, 1.3808e-01,
        1.0626e-01, 1.7575e-02, 1.1287e-02, 8.5349e-02, 3.4749e-02, 1.0000e+00,
        9.5889e-02, 1.4799e-01, 8.9803e-02, 1.5732e-01, 1.3144e-01, 9.6607e-02],
       dtype=torch.float64)

In [82]:
for i in range(0, 5):

    imput = torch.reshape(torch.tensor(X[i], dtype = torch.float32), (1,1,29))

    model.eval()

    print(model(imput, 1))

/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  taken = torch.take(batchX, construct_noOverlap_indices(torch.tensor(self.W, dtype = torch.float32), batchX.shape[0], self.W.shape[0]))
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
tensor([[281.5979]], grad_fn=<AddmmBackward>)
tensor([[282.0308]], grad_fn=<AddmmBackward>)
tensor([[279.2398]], grad_fn=<AddmmBackward>)
tensor([[281.1828]], grad_fn=<AddmmBackward>)
tensor([[266.7348]], grad_fn=<AddmmBackward

In [77]:
model.eval()

model(imput, 1)

/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  taken = torch.take(batchX, construct_noOverlap_indices(torch.tensor(self.W, dtype = torch.float32), batchX.shape[0], self.W.shape[0]))
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


tensor([[33225.4375]], grad_fn=<AddmmBackward>)

In [67]:
mig_cur = pd.read_csv("./us_migration.csv")
mig_cur = mig_cur.loc[:, ~mig_cur.columns.str.contains('^Unnamed')]
mig_cur = mig_cur.apply(lambda x: pd.to_numeric(x, errors='coerce'))
mig_cur = mig_cur.dropna(axis=1)
mig_cur = mig_cur.drop(["US_MIG_05_10"], axis = 1)

sending_index = mig_cur.index[mig_cur.sending == 10002]

mMScale = preprocessing.MinMaxScaler()
mMScale.fit(mig_cur)#.values


v = mMScale.transform([[100000005, 0, 0, 1141.005168, 191.67981899999998, 173.9693252, 143.1524397, 1668.6315789999999, 83.74568966, 112.0113636, 457.4188756, 849.3512360999999, 211.7425906, 212.12121209999998, 228.3500503, 4593.414458, 2690.181818, 366.9866667, 79.70120623, 200000000, 39760.0, 0, 0, 0.0, 0, 0, 512.0323625, 219.5713985, 197.11191340000002, 506.4814126]])


print(v)


imput = torch.reshape(torch.tensor(v, dtype = torch.float32), (1,1,30))

model(imput, 1)

# print(imput)

# mig_cur = mig_cur[mig_cur['sending'] == 1001]
# mig_cur = mig_cur.values




# imput = torch.reshape(torch.tensor(mig_cur, dtype = torch.float32), (1,1,30))

[[3.22037241e+03 1.32803178e-04 7.77725929e-05 1.66623865e-01
  2.42612691e-02 2.62448739e-03 1.32942023e-02 7.49160041e-02
  2.82476216e-03 1.03540152e-02 3.59565445e-02 1.11115362e-01
  3.63955388e-02 1.48865235e-03 5.73360792e-03 2.55398205e-01
  5.46895851e-02 3.16195618e-02 1.90831650e-02 2.93081402e-01
  7.52444535e-03 6.99953336e-05 4.13023757e-05 1.00000000e+00
  5.99964002e-05 1.31523086e-04 5.98497856e-02 2.85712445e-02
  1.34727059e-02 1.06247987e-01]]
/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  taken = torch.take(batchX, construct_noOverlap_indices(torch.tensor(self.W, dtype = torch.float32), batchX.shape[0], self.W.shape[0]))
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mod

tensor([[64188952.]], grad_fn=<AddmmBackward>)

In [33]:
imput

tensor([[[1.0010e+03, 2.6422e+03, 2.5049e+03, 2.4201e+03, 2.7446e+03,
          1.7018e+03, 2.5062e+03, 3.8308e+03, 1.6519e+03, 4.7719e+02,
          4.4098e+03, 2.6549e+03, 1.8496e+03, 3.1426e+03, 2.0203e+03,
          6.3189e+03, 7.1741e+03, 4.1580e+03, 1.1404e+03, 9.1146e+07,
          9.4033e+05, 2.3610e+03, 3.1542e+03, 0.0000e+00, 2.7246e+03,
          2.6439e+03, 1.9177e+03, 2.6851e+03, 1.7116e+03, 1.5648e+03]]])

In [34]:
model(imput, 1)

/Users/heatherbaier/Desktop/CAOE/sig/socialSig.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  taken = torch.take(batchX, construct_noOverlap_indices(torch.tensor(self.W, dtype = torch.float32), batchX.shape[0], self.W.shape[0]))
/Users/heatherbaier/anaconda/envs/caoe/lib/python3.6/site-packages/torch/nn/functional.py:3455: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)


tensor([[19946688.]], grad_fn=<AddmmBackward>)